In [2]:
import tensorflow as tf
import numpy as np

In [2]:
emb_encoder=a1
print(emb_encoder.shape.as_list())


encoder_outputs=[emb_encoder]
PAD_emb=tf.nn.embedding_lookup(emb,[1])

for enc_layer in range(6):
    with tf.variable_scope("encoder_%d"%enc_layer):
        emb_encoder=tf.pad(emb_encoder,paddings=[[0,0],[1,1],[0,0]],constant_values=PAD_emb)
        print(emb_encoder.shape.as_list())
        encoder_shape=emb_encoder.shape.as_list()

        emb_encoder=tf.reshape(emb_encoder,shape=[encoder_shape[0],encoder_shape[1],encoder_shape[2],1])# batch_size,seq_len,emb_dim,1

        filtersize=[3,10,1,20]
        filter=tf.Variable(initial_value=tf.truncated_normal(filtersize,stddev=0.01),name="filter")
        
        emb_encoder=tf.nn.conv2d(emb_encoder,filter=filter,strides=[1,1,1,1],padding="VALID",dilations=[1,1,1,1])
        emb_encoder=tf.reshape(emb_encoder,shape=[6,-1,10*2])
        A,B=tf.split(emb_encoder,2,axis=2)
        attn=tf.multiply(A,tf.nn.softmax(B))

        print(attn.shape.as_list())

        emb_encoder=attn+encoder_outputs[-1]
        a=tf.nn.softmax(emb_encoder,axis=1)
        print("a",a.shape.as_list())
        encoder_outputs.append(emb_encoder)
        print(emb_encoder.shape.as_list())



NameError: name 'a1' is not defined

In [6]:
a=np.array(range(48))
a=a.reshape([6,8])
a=tf.constant(a,dtype=tf.float32)
b=tf.constant([1,0,1,1,0,0],dtype=tf.float32,shape=[6,1])
g=tf.constant([[1,0,1,1,0,0],[1,0,1,1,0,0]],dtype=tf.float32,shape=[2,6])

c=tf.multiply(b,a)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
d,f=sess.run([c,tf.nn.l2_normalize(g,1)])
print(d)
print(f)

[[  0.   1.   2.   3.   4.   5.   6.   7.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [ 16.  17.  18.  19.  20.  21.  22.  23.]
 [ 24.  25.  26.  27.  28.  29.  30.  31.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]
[[ 0.57735026  0.          0.57735026  0.57735026  0.          0.        ]
 [ 0.57735026  0.          0.57735026  0.57735026  0.          0.        ]]


In [10]:
import tensorflow as tf 
a = tf.constant(1)
n = tf.constant(10)

def cond(a, n):
    return  a< n
def body(a,n):
    a = a + 1
    return a,n

a, n = tf.while_loop(cond, body, [a, n])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run([a,n])
    print(res)


[10, 10]


In [5]:
#词汇表中对应的主题词
a=tf.constant([[11,1],[12,2],[13,3],[14,4],[15,5],[16,6]],dtype=tf.int32)
# 词汇表
# b=tf.constant([[1,11,13],[3,16,6],[5,6,2]],dtype=tf.int32)
b=tf.constant([[0,13],[1,16],[0,8]],dtype=tf.int32)

emb=tf.get_variable(name='embb',shape=[20,5],dtype=tf.float32)
topic_emb=tf.get_variable(name='topic',shape=[10,5],dtype=tf.float32)

In [9]:
a_shape=a.get_shape()[0]
b_shape=b.get_shape()
size=b_shape[0]*b_shape[1]

def look_step(g,index,single,k1):
    t=tf.cond(tf.equal(single,a[k1,0]),
            lambda :(True,k1),
            lambda :(False,-1))
    return tf.logical_or(g,t[0]),tf.maximum(index,t[1]),single,k1+1
    
def _look(single):
    g=False
    k1=0
    index=-1
    g,index,*_=tf.while_loop(
        cond=lambda g,index,single,k1:k1<a_shape,
        body=look_step,
        loop_vars=[g,index,single,k1]
    )
    return g,index

k=tf.constant(1)
def loop_step(k,topic,matrix):
    g,index=_look(topic[k])
    matr=tf.cond(g,
                lambda :tf.gather(topic_emb,[a[index,1]]),
                lambda :tf.gather(emb,[topic[k]]))
    matr=tf.reshape(matr,[1,1,-1])
    matrix=tf.concat([matrix,matr],0)

    return k+1,topic,matrix

g=tf.reshape(b,[-1])
# matrix=tf.Variable([[]])
matrix=tf.nn.embedding_lookup(emb,[0])
matrix=tf.reshape(matrix,[1,1,-1])
_,_,matrix=tf.while_loop(
    cond=lambda k,b,*_:k<size,
    body=loop_step,
    loop_vars=[k,g,matrix],
    shape_invariants=[k.get_shape(),tf.TensorShape([None]),tf.TensorShape([None,1,5])]

)

matrix=tf.reshape(matrix,shape=[-1,b_shape[1],5])
print(matrix)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

matrix,topics,emm=sess.run([matrix,topic_emb,emb])
print(np.array(matrix).shape)
print(matrix)

print('------')
print(topics)
print('------')
print(emm)

Tensor("Reshape_11:0", shape=(?, 2, 5), dtype=float32)
(3, 2, 5)
[[[ 0.03646359 -0.35648412 -0.35819608 -0.07392752  0.20122376]
  [-0.05676717  0.4516198   0.05581731 -0.23014435  0.16458923]]

 [[ 0.3693814   0.21581456  0.08683088  0.21898589 -0.24347787]
  [-0.15781456 -0.41223702 -0.14436388  0.50856084  0.02451092]]

 [[ 0.03646359 -0.35648412 -0.35819608 -0.07392752  0.20122376]
  [-0.35203528  0.20256558 -0.05344453  0.36638007 -0.4004116 ]]]
------
[[ 0.47056848  0.09352803 -0.5762763  -0.31750795 -0.03069645]
 [-0.06277215  0.14072305 -0.11286432 -0.34434542  0.1345641 ]
 [-0.26202917  0.26201773  0.03084528  0.26555747  0.4731199 ]
 [-0.05676717  0.4516198   0.05581731 -0.23014435  0.16458923]
 [ 0.07173115  0.22346711 -0.17219594  0.37617642 -0.02289522]
 [-0.08737779  0.6083922  -0.29488206 -0.02280551 -0.5336372 ]
 [-0.15781456 -0.41223702 -0.14436388  0.50856084  0.02451092]
 [-0.5874706  -0.34369203  0.6219401   0.57480067 -0.40176803]
 [-0.06785178 -0.3215666  -0.52732

In [4]:
i = tf.constant(0)
l = tf.Variable([])
array = tf.Variable(tf.random_normal([10]))
def body(i, l):                                               
    temp = tf.gather(array,i)
    l = tf.concat([l, [temp]], 0)
    return i+2, l
def cond(i,l):
   return i < 10
index, list_vals = tf.while_loop(cond, body, [i, l],shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None])])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
a1,b1=sess.run([list_vals,array])
print(a1)
print(b1)


[ 0.36802626 -0.19462073 -0.8859022  -0.5048751   0.5068357 ]
[ 0.36802626  0.11136688 -0.19462073 -0.11375644 -0.8859022  -0.4814587
 -0.5048751   0.9559712   0.5068357   1.625209  ]


In [26]:
gh1=tf.gather(emb,[1])
gh2=tf.gather(emb,[0])
gh=tf.concat([gh2,gh1],axis=0)
print(gh)
a1,b1=sess.run([emb,gh])


Tensor("concat_4:0", shape=(2, 5), dtype=float32)
[[-0.54136574  0.03651583 -0.27027145  0.06238627  0.16824466]
 [-0.44653267 -0.12659621  0.02578914 -0.01459128 -0.09342879]
 [-0.17026371 -0.5374914  -0.05861431 -0.17510164  0.20600271]
 [ 0.00965261  0.06737155  0.6241546  -0.27840954  0.5559228 ]
 [ 0.12267345 -0.08674705  0.35988408 -0.60730344  0.43452126]
 [ 0.12683946  0.10327429 -0.03427905  0.5521496  -0.35221782]
 [-0.1526998   0.1997881  -0.23401058 -0.34670693 -0.10232884]
 [ 0.44554812  0.0498898  -0.5839828  -0.12379533  0.1324259 ]
 [-0.06297046 -0.27502432 -0.03852886 -0.01656979  0.60367066]
 [ 0.19230837 -0.38089323 -0.04958206  0.37696415  0.34378386]]
______
[[-0.54136574  0.03651583 -0.27027145  0.06238627  0.16824466]
 [-0.44653267 -0.12659621  0.02578914 -0.01459128 -0.09342879]]


In [25]:

a=np.array(range(24))

a = tf.reshape(tf.constant(a),shape=[2,3,4])
aa=tf.argmax(a,-1)
print(aa)
print(sess.run(tf.concat(aa,1)))

Tensor("ArgMax_20:0", shape=(2, 3), dtype=int64)
[[3 3 3]
 [3 3 3]]
